<a href="https://colab.research.google.com/github/gnitnaw/LDL/blob/main/tf_framework/c11e1_autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


'\nThe MIT License (MIT)\nCopyright (c) 2021 NVIDIA\nPermission is hereby granted, free of charge, to any person obtaining a copy of\nthis software and associated documentation files (the "Software"), to deal in\nthe Software without restriction, including without limitation the rights to\nuse, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of\nthe Software, and to permit persons to whom the Software is furnished to do so,\nsubject to the following conditions:\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS\nFOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR\nCOPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER\nIN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OU

This code example demonstrates how to use an LSTM-based neural network and beam search to do text autocompletion. More context for this code example can be found in the section "Programming Example: Using LSTM for Text Autocompletion" in Chapter 11 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).


The initialization code is shown in the first code snippet. Apart from the import statements, we need to provide the path to the text file to use for training. We also define two variables, WINDOW_LENGTH and WINDOW_STEP, which are used to control the process of splitting up this text file into multiple training examples. The other three variables control the beam-search algorithm and are described shortly. The text used to train the model is assumed to be in the file ../data/frankenstein.txt.

# Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path_head = '/content/drive/MyDrive/Colab Notebooks/' # You have to change this. 

Mounted at /content/drive


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 32
BATCH_SIZE = 256
#INPUT_FILE_NAME = '../data/frankenstein.txt'
INPUT_FILE_NAME = path_head+'data/frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
BEAM_SIZE = 8
NUM_LETTERS = 11
MAX_LENGTH = 50


The next code snippet opens and reads the content of the file, converts it all into lowercase, and replaces double spaces with single spaces. To enable us to easily one-hot encode each character, we want to assign a monotonically increasing index to each character. This is done by first creating a list of unique characters. Once we have that list, we can loop over it and assign an incrementing index to each character. We do this twice to create one dictionary (a hash table) that maps from character to index and a reverse dictionary from index to character. These will come in handy later when we want to convert text into one-hot encoded input to the network as well as when we want to convert one-hot encoded output into characters. Finally, we initialize a variable encoding_width with the count of unique characters, which will be the width of each one-hot encoded vector that represents a character.

In [4]:
# Open the input file.
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8-sig')
text = file.read()
file.close()

# Make lowercase and remove newline and extra spaces.
text = text.lower()
text = text.replace('\n', ' ')
text = text.replace('  ', ' ')

# Encode characters as indices.
unique_chars = list(set(text))
char_to_index = dict((ch, index) for index,
                     ch in enumerate(unique_chars))
index_to_char = dict((index, ch) for index,
                     ch in enumerate(unique_chars))
encoding_width = len(char_to_index)


The next step is to create training examples from the text file. This is done by the next code snippet. Each training example will consist of a sequence of characters and a target output value of a single character immediately following the input characters. We create these input examples using a sliding window of length WINDOW_LENGTH. Once we have created one training example, we slide the window by WINDOW_STEP positions and create the next training example. We add the input examples to one list and the output values to another. All of this is done by the first for loop.

We then create a single tensor holding all the input examples and another tensor holding the output values. Both of these tensors will hold data in one-hot encoded form, so each character is represented by a dimension of size encoding_width. We first allocate space for the two tensors and then fill in the values using a nested for loop.


In [5]:
# Create training examples.
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])

# Convert to one-hot encoded training data.
X = np.zeros((len(fragments), WINDOW_LENGTH, encoding_width))
y = np.zeros((len(fragments), encoding_width))
for i, fragment in enumerate(fragments):
    for j, char in enumerate(fragment):
        X[i, j, char_to_index[char]] = 1
    target_char = targets[i]
    y[i, char_to_index[target_char]] = 1


We are now ready to build our model. From the perspective of training our model, it will look similar to the book sales prediction example, but we use a deeper model consisting of two LSTM layers. Both LSTM layers use a dropout value of 0.2 on the connections between layers as well as on the recurrent connections. Note how we pass return_sequences=True to the constructor of the first layer because the second layer needs to see the output values for all timesteps from the first layer. The second LSTM layer is followed by a fully connected layer with multiple neurons using a softmax function because we will be predicting probabilities for discrete entities (characters). We use categorical cross-entropy as our loss function, which is the recommended loss function for multicategory classification.

One thing to note is that when we prepared the data, we did not split the dataset into a training set and a test set. Instead, we provide a parameter validation_ split=0.05 to the fit() function. Keras will then automatically split our training data into a training set and a test set, where the parameter 0.05 indicates that 5% of the data will be used as a test set. We will also inspect the predictions to get an idea of whether the network is doing what we would like it to do. Finally, we train the model for 32 epochs with a mini-batch size of 256.


In [6]:
# Build and train model.
model = Sequential()
model.add(LSTM(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2,
                        input_shape=(None, encoding_width)))
model.add(LSTM(128, dropout=0.2,
                        recurrent_dropout=0.2))
model.add(Dense(encoding_width, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam')
model.summary()
history = model.fit(X, y, validation_split=0.05,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS, verbose=2,
                    shuffle=True)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 128)         99840     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 66)                8514      
                                                                 
Total params: 239,938
Trainable params: 239,938
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
542/542 - 169s - loss: 2.7819 - val_loss: 2.6343 - 169s/epoch - 312ms/step
Epoch 2/32
542/542 - 163s - loss: 2.3966 - val_loss: 2.4968 - 163s/epoch - 302ms/step
Epoch 3/32
542/542 - 165s - loss: 2.2670 - val_loss: 2.4290 - 165s/epoch - 305ms/step
Epoch 4/32
542/542 - 164s - loss: 2.1824 - val_loss: 2.3

The next step is to implement the beamsearch algorithm to predict text. Consult the relevant section in the book for more details about beamsearch. In our implementation, each beam is represented by a tuple with three elements. The first element is the logarithm of the cumulative probability for the current sequence of characters (logarithm is used to avoid numerical underflow, see book for more details). The second element is the string of characters. The third element is a one-hot encoded version of the string of characters.

We start by creating a single beam with an initial sequence of characters ('the body ') and set the initial probability to 1.0. The one-hot encoded version of the string is created by the first loop. We add this beam to a list named beams.

This is followed by a nested loop that uses the trained model to do predictions according to the beam-search algorithm. We extract the one-hot encoding representation of each beam and create a NumPy array with multiple input examples. There is one input example per beam. During the first iteration, there is only a single input example. During the remaining iterations, there will be BEAM_SIZE number of examples.

We call model.predict(), which results in one softmax vector per beam. The softmax vector contains one probability per word in the vocabulary. For each beam, we create BEAM_SIZE new beams, each beam consisting of the words from the original beam concatenated with one more word. We choose the most probable words when creating the beams. The probability for each beam can be computed by multiplying the current probability of the beam by the probability for the added word.

Once we have created BEAM_SIZE beams for each existing beam, we sort the list of new beams according to their probabilities. We then discard all but the top BEAM_SIZE beams. This represents the pruning step. For the first iteration, this does not result in any pruning because we started with a single beam, and this beam resulted in just BEAM_SIZE beams. For all remaining iterations, we will end up with BEAM_SIZE * BEAM_SIZE beams and discard most of them.

The loop runs for a fixed number of iterations followed by printing out the generated predictions.


In [7]:
# Create initial single beam represented by triplet
# (probability , string , one-hot encoded string).
letters = 'the body '
one_hots = []
for i, char in enumerate(letters):
    x = np.zeros(encoding_width)
    x[char_to_index[char]] = 1
    one_hots.append(x)
beams = [(np.log(1.0), letters, one_hots)]

# Predict NUM_LETTERS into the future.
for i in range(NUM_LETTERS):
    minibatch_list = []
    # Create minibatch from one-hot encodings, and predict.
    for triple in beams:
        minibatch_list.append(triple[2])
    minibatch = np.array(minibatch_list)
    y_predict = model.predict(minibatch, verbose=0)
    new_beams = []
    for j, softmax_vec in enumerate(y_predict):
        triple = beams[j]
        # Create BEAM_SIZE new beams from each existing beam.
        for k in range(BEAM_SIZE):
            char_index = np.argmax(softmax_vec)
            new_prob = triple[0] + np.log(
                softmax_vec[char_index])
            new_letters = triple[1] + index_to_char[char_index]
            x = np.zeros(encoding_width)
            x[char_index] = 1
            new_one_hots = triple[2].copy()
            new_one_hots.append(x)
            new_beams.append((new_prob, new_letters,
                              new_one_hots))
            softmax_vec[char_index] = 0
    # Prune tree to only keep BEAM_SIZE most probable beams.
    new_beams.sort(key=lambda tup: tup[0], reverse=True)
    beams = new_beams[0:BEAM_SIZE]
for item in beams:
    print(item[1])


the body of the pres
the body of the comp
the body of the mind
the body of the more
the body of the coul
the body of the sump
the body of the coun
the body of the mort
